In [ ]:
using LowLevelFEM
import LowLevelFEM as FEM
gmsh.initialize()

In [ ]:
gmsh.open("body2.geo")

In [ ]:
mat = FEM.material("body")
problem = FEM.Problem([mat])

In [ ]:
supp = FEM.displacementConstraint("left", ux=0, uy=0, uz=0)
load = FEM.load("right", fx=1)

In [ ]:
K = FEM.stiffnessMatrix(problem)
f = FEM.loadVector(problem, [load])

In [ ]:
FEM.applyBoundaryConditions!(problem, K, f, [supp])

In [ ]:
q1 = FEM.solveDisplacement(K, f)

In [ ]:
FEM.showDoFResults(problem, q1, :uvec, visible=true)

In [ ]:
q2 = FEM.solveDisplacement(problem, [load], [supp])

In [ ]:
FEM.showDoFResults(problem, q2, :uvec)

In [ ]:
elsupp = FEM.elasticSupport("left", kx=1e6, ky=1e6, kz=1e6)

In [ ]:
q4 = FEM.solveDisplacement(problem, [load], [], [elsupp])

In [ ]:
FEM.showDoFResults(problem, q4, :uvec)

In [ ]:
q3 = copy(q2)
q3.a[1] = 5
q2.a[1] = 3
q3.a[1]

In [ ]:
q2 + q3
q2 - q3
q2 * 2
2 * q2
q2 / 2

In [ ]:
A1 = FEM.solveStrain(problem, q1)

In [ ]:
A2 = FEM.solveStrain(problem, q1, DoFResults=true)

In [ ]:
FEM.showStrainResults(problem, A1, :e)

In [ ]:
FEM.showDoFResults(problem, A2, :e)

In [ ]:
A3 = FEM.elementsToNodes(problem, A1)

In [ ]:
FEM.showDoFResults(problem, A3, :e)

In [ ]:
S1 = FEM.solveStress(problem, q1)

In [ ]:
S2 = FEM.solveStress(problem, q1, DoFResults=true)

In [ ]:
FEM.showStressResults(problem, S1, :s)

In [ ]:
FEM.showDoFResults(problem, S2, :s)

In [ ]:
S3 = FEM.elementsToNodes(problem, S1)

In [ ]:
FEM.showDoFResults(problem, S3, :s)

In [ ]:
thermo = FEM.Problem([mat], type=:HeatConduction)

In [ ]:
temp = FEM.temperatureConstraint("left", T=0)
flux = FEM.heatFlux("right", qn=1)

flx = FEM.heatFluxVector(thermo, [flux])

In [ ]:
KT = FEM.heatConductionMatrix(thermo)

FEM.applyBoundaryConditions!(thermo, KT, flx, [temp])

T1 = FEM.solveTemperature(KT, flx)

In [ ]:
FEM.showDoFResults(thermo, T1, :T)

In [ ]:
T2 = FEM.solveTemperature(thermo, [flux], [temp])

In [ ]:
FEM.showDoFResults(thermo, T2, :T)

In [ ]:
qT1 = FEM.solveHeatFlux(thermo, T2)

In [ ]:
FEM.showElementResults(thermo, qT1, :qvec)

In [ ]:
qT2 = FEM.solveHeatFlux(thermo, T2, DoFResults=true)

In [ ]:
FEM.showDoFResults(thermo, qT2, :qvec)

In [ ]:
qT3 = FEM.elementsToNodes(thermo, qT1)

In [ ]:
FEM.showDoFResults(thermo, qT3, :qvec)

In [ ]:
heatconvB = FEM.heatConvection("bottom", h=1, Tₐ=0)
heatconvT = FEM.heatConvection("top", h=100, Tₐ=20)
heatconvF = FEM.heatConvection("front", h=1, Tₐ=0)
heatconvR = FEM.heatConvection("rear", h=1, Tₐ=0)

In [ ]:
T3 = FEM.solveTemperature(thermo, [], [temp], [heatconvB, heatconvT, heatconvF, heatconvR])

In [ ]:
FEM.showDoFResults(thermo, T3, :T)

In [ ]:
FEM.openPostProcessor()

In [ ]:
gmsh.finalize()